In [30]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

from textblob import TextBlob
 
import twitter_credentials

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import csv


# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user, tweet_mode="extended").items(num_tweets):
                tweets.append(tweet)
        return tweets



# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        #stream = Stream(auth, listener)
        stream = Stream(auth, listener, tweet_mode='extended')
        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """


    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
           return -1

    def tweets_to_data_frame(self, tweets):
       #  maybe just go straight to csv
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['date'] = np.array([tweet.created_at for tweet in tweets])
    

        return df

 
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="seldo", count=15)
    print(tweets)
    
    full_tweets = []
    for tweet in tweets:
        full_tweets.append(tweet.text)
    #print(full_tweets)
    
    full_tweets1 = np.array(full_tweets)

    with open('full_seldo_tweeets.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for row in range(0,full_tweets1.shape[0]):
            myList = []
            myList.append(full_tweets1[row])
            writer.writerow(myList) 
    
    
   # for self._tweet in tweepy.Cursor(self._api.search,q=self._screen_name,count=300, lang="en", tweet_mode="extended").items(300):
   # self._csvWriter.writerow([self._tweet.created_at, self._tweet.full_text.encode('utf-8')])
    
    
    df = tweet_analyzer.tweets_to_data_frame(tweets)
    #df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])
    
  # df.to_csv("seldo_tweets", sep='\n')
    #check if that removes tweet length
   # print(df.head(10))


[Status(_api=<tweepy.api.API object at 0x1a1b338e80>, _json={'created_at': 'Wed Apr 24 00:00:32 +0000 2019', 'id': 1120839905041510400, 'id_str': '1120839905041510400', 'text': '@jchiatt @BenLesh I can still give you recs and intro you to some people!', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'jchiatt', 'name': 'J.C. Hiatt ❁', 'id': 42551947, 'id_str': '42551947', 'indices': [0, 8]}, {'screen_name': 'BenLesh', 'name': 'Ben Lesh 🧢🏋️\u200d♂️💻🎨', 'id': 23795212, 'id_str': '23795212', 'indices': [9, 17]}], 'urls': []}, 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'in_reply_to_status_id': 1120839463092072448, 'in_reply_to_status_id_str': '1120839463092072448', 'in_reply_to_user_id': 42551947, 'in_reply_to_user_id_str': '42551947', 'in_reply_to_screen_name': 'jchiatt', 'user': {'id': 15453, 'id_str': '15453', 'name': 'Laurie Voss', 'screen_name': 'seldo', 'location': 'San Francisco, CA', 'description